In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Bidirectional, Dropout, Dense
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
from utils import load_nusax_data
from model import ManualLSTMModel

In [2]:
# ----- Fixed Hyperparameters -----
MAX_TOKENS    = 20000
SEQ_LENGTH    = 100
EMBED_DIM     = 128
DROPOUT_RATE  = 0.5
BATCH_SIZE    = 50
EPOCHS        = 15

# ----- Load and preprocess data -----
tok_train, y_train, tok_val, y_val, tok_test, y_test, vocab_size, num_classes, vectorizer = \
    load_nusax_data(MAX_TOKENS, SEQ_LENGTH)

os.makedirs('results/lstm_weights', exist_ok=True)
os.makedirs('results/lstm_plots', exist_ok=True)
results_summary_lstm = []

In [5]:
# ----- Needed Function Definitions -----
def build_keras_lstm_model(vocab_size_fn, embedding_dim_fn, num_lstm_layers, lstm_units_fn, bidirectional_fn, num_classes_fn, dropout_rate_fn, seq_length_fn):
    keras_model = Sequential(name=f"keras_lstm_{num_lstm_layers}L_{lstm_units_fn}U_{'Bi' if bidirectional_fn else 'Uni'}")
    keras_model.add(Embedding(
        input_dim=vocab_size_fn,
        output_dim=embedding_dim_fn,
        input_length=seq_length_fn,
        name='embedding'
    ))

    for i in range(num_lstm_layers):
        is_last_lstm = (i == num_lstm_layers - 1)
        return_sequences = not is_last_lstm

        layer_name_prefix = f"{'bi' if bidirectional_fn else ''}lstm_{i}"
        lstm_layer_constructor = LSTM(lstm_units_fn, return_sequences=return_sequences, name=f"{layer_name_prefix}_lstm")

        if bidirectional_fn:
            keras_model.add(Bidirectional(lstm_layer_constructor, name=f"{layer_name_prefix}_bidir"))
        else:
            keras_model.add(lstm_layer_constructor)
            
    keras_model.add(Dropout(dropout_rate_fn, name='dropout'))
    keras_model.add(Dense(num_classes_fn, activation='softmax', name='output'))

    keras_model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return keras_model

def plot_history(history, model_name_fn, experiment_name_fn):
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title(f'Loss - {model_name_fn}')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title(f'Accuracy - {model_name_fn}')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    
    plot_filename = f"results/lstm_plots/{experiment_name_fn}_{model_name_fn}_history.png"
    os.makedirs(os.path.dirname(plot_filename), exist_ok=True)
    plt.savefig(plot_filename)
    print(f"Saved plot to {plot_filename}")
    plt.close()

In [6]:
# --- Experiment Configurations for LSTM ---
experiments_lstm_config = []

# A. Pengaruh jumlah layer LSTM
lstm_units_A = 64
bidirectional_A = True
dropout_A = DROPOUT_RATE
for num_layers in [1, 2, 3]:
    experiments_lstm_config.append({
        "name": f"LSTM_NumLayers_{num_layers}_Units_{lstm_units_A}_Dir_{'Bi' if bidirectional_A else 'Uni'}",
        "num_lstm_layers": num_layers,
        "lstm_units": lstm_units_A,
        "bidirectional": bidirectional_A,
        "dropout_rate": dropout_A,
        "compare_custom": (num_layers == 1 and bidirectional_A)
    })

# B. Pengaruh banyak cell LSTM per layer
num_layers_B = 1
bidirectional_B = True
dropout_B = DROPOUT_RATE
for lstm_units_val in [32, 64, 128]:
    experiments_lstm_config.append({
        "name": f"LSTM_NumLayers_{num_layers_B}_Units_{lstm_units_val}_Dir_{'Bi' if bidirectional_B else 'Uni'}",
        "num_lstm_layers": num_layers_B,
        "lstm_units": lstm_units_val,
        "bidirectional": bidirectional_B,
        "dropout_rate": dropout_B,
        "compare_custom": (num_layers_B == 1 and bidirectional_B and lstm_units_val == 64)
    })

# C. Pengaruh jenis layer LSTM berdasarkan arah
num_layers_C = 1
lstm_units_C = 64
dropout_C = DROPOUT_RATE
for bidirectional_flag in [False, True]:
    experiments_lstm_config.append({
        "name": f"LSTM_NumLayers_{num_layers_C}_Units_{lstm_units_C}_Dir_{'Bi' if bidirectional_flag else 'Uni'}",
        "num_lstm_layers": num_layers_C,
        "lstm_units": lstm_units_C,
        "bidirectional": bidirectional_flag,
        "dropout_rate": dropout_C,
        "compare_custom": (num_layers_C == 1)
    })

In [7]:
# --- Run LSTM Experiments ---
for i, exp_config in enumerate(experiments_lstm_config):
    print(f"\n--- Running LSTM Experiment {i+1}/{len(experiments_lstm_config)}: {exp_config['name']} ---")
    
    model_name = exp_config['name']
    
    keras_lstm_model = build_keras_lstm_model(
        vocab_size_fn=vocab_size,
        embedding_dim_fn=EMBED_DIM,
        num_lstm_layers=exp_config['num_lstm_layers'],
        lstm_units_fn=exp_config['lstm_units'],
        bidirectional_fn=exp_config['bidirectional'],
        num_classes_fn=num_classes,
        dropout_rate_fn=exp_config['dropout_rate'],
        seq_length_fn=SEQ_LENGTH
    )
    keras_lstm_model.summary()

    print(f"Training Keras LSTM model: {model_name}")
    history = keras_lstm_model.fit(
        tok_train, y_train,
        validation_data=(tok_val, y_val),
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        verbose=2
    )
    
    weights_path = f'results/lstm_weights/keras_lstm_{model_name}.weights.h5'
    keras_lstm_model.save_weights(weights_path)
    print(f"Saved Keras LSTM weights to {weights_path}")

    plot_history(history, model_name, "lstm_experiment")

    y_pred_prob_keras = keras_lstm_model.predict(tok_test, batch_size=BATCH_SIZE)
    y_pred_keras = np.argmax(y_pred_prob_keras, axis=1)
    f1_test_keras = f1_score(y_test, y_pred_keras, average='macro')
    print(f"[Keras LSTM - {model_name}] Test Macro F1-score: {f1_test_keras:.4f}")
    
    current_results = {
        "name": model_name,
        "num_lstm_layers": exp_config['num_lstm_layers'],
        "lstm_units": exp_config['lstm_units'],
        "bidirectional": exp_config['bidirectional'],
        "f1_keras": f1_test_keras,
        "f1_custom": None 
    }

    if exp_config.get("compare_custom", False):
        print(f"\n--- Comparing with Manual LSTM Model for {model_name} ---")
        
        # ManualLSTMModel Anda di LSTMmodel.py dirancang untuk 1 layer BiLSTM.
        # Jadi, kita hanya bisa membandingkan jika konfigurasi Keras cocok.
        if exp_config['num_lstm_layers'] == 1 and exp_config['bidirectional']:
            manual_model = ManualLSTMModel(
                vocab_size=vocab_size,
                embedding_dim=EMBED_DIM,
                lstm_units=exp_config['lstm_units'], # Gunakan lstm_units dari config
                dropout_rate=exp_config['dropout_rate'],
                num_classes=num_classes
            )
            
            try:
                manual_model.load_weights(weights_path) # Memuat dari file .h5
                print("Loaded Keras weights into manual LSTM model.")

                manual_model.dropout.set_training_mode(False) # Set mode evaluasi
                
                probs_manual = manual_model.forward(tok_test, training=False)
                preds_manual = np.argmax(probs_manual, axis=1)
                f1_manual = f1_score(y_test, preds_manual, average='macro')
                print(f"[Manual LSTM - {model_name}] Test Macro F1-score: {f1_manual:.4f}")
                current_results["f1_custom"] = f1_manual

                if not np.isclose(f1_test_keras, f1_manual, atol=1e-3): # Toleransi bisa disesuaikan
                    print("Warning: Keras LSTM and Manual LSTM F1 scores differ significantly.")
            except Exception as e:
                print(f"Error during manual LSTM model comparison for {model_name}: {e}")
                print("Please ensure 'ManualLSTMModel.load_weights' correctly parses the Keras H5 file structure for this configuration.")
        else:
            print(f"Skipping custom LSTM model comparison for {model_name} as its Keras architecture "
                  f"({exp_config['num_lstm_layers']} layer(s), "
                  f"{'Bi' if exp_config['bidirectional'] else 'Uni'}) "
                  "does not match the expected 1-layer Bidirectional LSTM structure of ManualLSTMModel.")
    
    results_summary_lstm.append(current_results)


--- Running LSTM Experiment 1/8: LSTM_NumLayers_1_Units_64_Dir_Bi ---


c:\Python312\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "keras_lstm_1L_64U_Bi"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bilstm_0_bidir (Bidirectional)  │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output (Dense)                  │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bilstm_0_bidir (Bidirectional)  │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output (Dense)                  │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Training Keras LSTM model: LSTM_NumLayers_1_Units_64_Dir_Bi
Epoch 1/15
10/10 - 3s - 312ms/step - accuracy: 0.3980 - loss: 1.0839 - val_accuracy: 0.4000 - val_loss: 1.0655
Epoch 2/15
10/10 - 1s - 75ms/step - accuracy: 0.4740 - loss: 1.0317 - val_accuracy: 0.4900 - val_loss: 1.0038
Epoch 3/15
10/10 - 1s - 74ms/step - accuracy: 0.6020 - loss: 0.9165 - val_accuracy: 0.5800 - val_loss: 0.9036
Epoch 4/15
10/10 - 1s - 72ms/step - accuracy: 0.6760 - loss: 0.7534 - val_accuracy: 0.6200 - val_loss: 0.8276
Epoch 5/15
10/10 - 1s - 80ms/step - accuracy: 0.7500 - loss: 0.5915 - val_accuracy: 0.6800 - val_loss: 0.7412
Epoch 6/15
10/10 - 1s - 75ms/step - accuracy: 0.8480 - loss: 0.4306 - val_accuracy: 0.6700 - val_loss: 0.7468
Epoch 7/15
10/10 - 1s - 72ms/step - accuracy: 0.8980 - loss: 0.3173 - val_accuracy: 0.6800 - val_loss: 0.7778
Epoch 8/15
10/10 - 1s - 74ms/step - accuracy: 0.9540 - loss: 0.2133 - val_accuracy: 0.8100 - val_loss: 0.6467
Epoch 9/15
10/10 - 1s - 74ms/step - accuracy: 0.9880 - loss

In [8]:
# --- Print LSTM Experiment Results Summary ---
print("\n\n--- LSTM Experiment Results Summary ---")
print("Name | LSTM Layers | LSTM Units | Bidirectional | F1 Keras | F1 Custom")
print("-" * 100) # Disesuaikan agar lebih lebar
for res in results_summary_lstm:
    f1_custom_str = f"{res['f1_custom']:.4f}" if res['f1_custom'] is not None else "N/A"
    print(f"{res['name']:<60} | {res['num_lstm_layers']:^11} | {res['lstm_units']:^10} | {str(res['bidirectional']):<13} | {res['f1_keras']:.4f}   | {f1_custom_str}")



--- LSTM Experiment Results Summary ---
Name | LSTM Layers | LSTM Units | Bidirectional | F1 Keras | F1 Custom
----------------------------------------------------------------------------------------------------
LSTM_NumLayers_1_Units_64_Dir_Bi                             |      1      |     64     | True          | 0.7349   | 0.7349
LSTM_NumLayers_2_Units_64_Dir_Bi                             |      2      |     64     | True          | 0.7151   | N/A
LSTM_NumLayers_3_Units_64_Dir_Bi                             |      3      |     64     | True          | 0.6753   | N/A
LSTM_NumLayers_1_Units_32_Dir_Bi                             |      1      |     32     | True          | 0.7604   | N/A
LSTM_NumLayers_1_Units_64_Dir_Bi                             |      1      |     64     | True          | 0.7178   | 0.7178
LSTM_NumLayers_1_Units_128_Dir_Bi                            |      1      |    128     | True          | 0.7417   | N/A
LSTM_NumLayers_1_Units_64_Dir_Uni                     

## ACCURACRY TEST (F1)

In [24]:
vocab_size     = 20000   # size of your word index
embedding_dim  = 128     # dimensionality of the embedding vectors
lstm_units     = 64      # number of LSTM units
dropout_rate   = 0.5     # dropout probability
num_classes    = 3       # number of target classes

# 2) Build the model
model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=embedding_dim),
    Bidirectional(LSTM(units=lstm_units)),
    Dropout(rate=dropout_rate),
    Dense(units=num_classes, activation='softmax')
])

# 3) Compile
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

# Train
BATCH_SIZE    = 50
EPOCHS        = 15

history = model.fit(
        tok_train, y_train,
        validation_data=(tok_val, y_val),
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        verbose=2
    )

model.save_weights("model_comp.weights.h5")

Epoch 1/15
10/10 - 3s - 335ms/step - accuracy: 0.4060 - loss: 1.0786 - val_accuracy: 0.4400 - val_loss: 1.0623
Epoch 2/15
10/10 - 1s - 56ms/step - accuracy: 0.4920 - loss: 1.0130 - val_accuracy: 0.4700 - val_loss: 1.0036
Epoch 3/15
10/10 - 1s - 57ms/step - accuracy: 0.5780 - loss: 0.9141 - val_accuracy: 0.5500 - val_loss: 0.9248
Epoch 4/15
10/10 - 1s - 60ms/step - accuracy: 0.6660 - loss: 0.7910 - val_accuracy: 0.6000 - val_loss: 0.8693
Epoch 5/15
10/10 - 1s - 57ms/step - accuracy: 0.7380 - loss: 0.6637 - val_accuracy: 0.6500 - val_loss: 0.7791
Epoch 6/15
10/10 - 1s - 59ms/step - accuracy: 0.8640 - loss: 0.4712 - val_accuracy: 0.7000 - val_loss: 0.6747
Epoch 7/15
10/10 - 1s - 60ms/step - accuracy: 0.9300 - loss: 0.2868 - val_accuracy: 0.7100 - val_loss: 0.6991
Epoch 8/15
10/10 - 1s - 61ms/step - accuracy: 0.8980 - loss: 0.2972 - val_accuracy: 0.6600 - val_loss: 0.8541
Epoch 9/15
10/10 - 1s - 55ms/step - accuracy: 0.9240 - loss: 0.2431 - val_accuracy: 0.6500 - val_loss: 0.7960
Epoch 10/

In [25]:
manual = ManualLSTMModel(vocab_size, embedding_dim,lstm_units, dropout_rate, num_classes)
manual.load_weights("model_comp.weights.h5")

In [26]:
prediction = model.predict(tok_test)
print("Manual pred :")
manual_prediction = manual.forward(tok_test)

13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step
Manual pred :


In [27]:
import numpy as np
from sklearn.metrics import f1_score, classification_report

pred_labels = np.argmax(prediction, axis=1)
manual_pred_labels = np.argmax(manual_prediction, axis=1)

f1_source = f1_score(y_test, pred_labels, average='macro')
f1_manual = f1_score(y_test, manual_pred_labels, average='macro')

print(f"F1 Bawaan:    {f1_source:.4f}")
print(classification_report(y_test, pred_labels, digits=4))
print("==========")
print("==========")
print("==========\n")
print(f"F1 Buatan:    {f1_manual:.4f}")
print(classification_report(y_test, manual_pred_labels, digits=4))

F1 Bawaan:    0.7203
              precision    recall  f1-score   support

           0     0.7122    0.6471    0.6781       153
           1     0.6915    0.6771    0.6842        96
           2     0.7605    0.8411    0.7987       151

    accuracy                         0.7275       400
   macro avg     0.7214    0.7217    0.7203       400
weighted avg     0.7255    0.7275    0.7251       400


F1 Buatan:    0.7203
              precision    recall  f1-score   support

           0     0.7122    0.6471    0.6781       153
           1     0.6915    0.6771    0.6842        96
           2     0.7605    0.8411    0.7987       151

    accuracy                         0.7275       400
   macro avg     0.7214    0.7217    0.7203       400
weighted avg     0.7255    0.7275    0.7251       400

